# Introduction
This following Python program aims to rearrange the given folder and its subfolders in ordered way.My intention for creating this program is that I have a large collection of Videos and some of them are in folers others are without any folder, some program have subtitle and there are duplicate Videos present also there is a problem with the name of the files. So I'm Writing this program so that every file have a correct name and folder with subtitle file with the same name. I will also try to prepare a log file for the activities tha are being performed and missing srt(subtitle file).

#1 first of all the program ask the use for the directory using browse option or any other method
#2 read the file structure and the metadata
#3 make folders for files with no folder
#4 move the intended files to their respective folders
#5 check for subtitle files along with the reading of file and metadata using the extension of the file
#6 make a log file listing all the unavailable subtitle files
#7 make a log file along with the messages while moving the files to their respective folders.

In [1]:
import sys
import os